In [15]:
import os
import json
from time import sleep

import pandas as pd
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import RandomNodeSplit
import random
import numpy as np
from collections import defaultdict
from src.shared.database_wrapper import DatabaseWrapper
from src.datasets.who_is_who import WhoIsWhoDataset
from src.model.GAT.gat_encoder import GATv2Encoder
from src.model.GAT.gat_decoder import GATv2Decoder
from src.shared.graph_schema import NodeType, EdgeType, node_one_hot, edge_one_hot
from src.shared import config

import networkx as nx
import plotly.graph_objects as go

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [16]:
driver = GraphDatabase.driver(config.DB_URI, auth=(config.DB_USER, config.DB_PASSWORD))

In [17]:
def fetch_n_hop_neighbourhood(
        start_node_type: NodeType, 
        start_node_id: str,
        node_attr: str, 
        node_types: list = None, 
        edge_types: list = None,
        max_level: int = 6
):
    with driver.session() as session:
        node_filter = '|'.join(
            [nt.value for nt in NodeType] if node_types is None else 
            [nt.value for nt in node_types]
        )
        edge_filter = '|'.join(
            [f"<{et.value}" for et in EdgeType] if edge_types is None else 
            [f"<{et.value}" for et in edge_types]
        )
        
        query = f"""
                MATCH (start:{start_node_type.value} {{id: '{start_node_id}'}})
                CALL apoc.path.subgraphAll(start, {{
                  maxLevel: {max_level},
                  relationshipFilter: '{edge_filter}',
                  labelFilter: '+{node_filter}'
                }}) YIELD nodes, relationships
                RETURN nodes, relationships
            """
        result = session.run(query)
        data = result.single()
        if not data:
            return None, None, None
        
        nodes = data["nodes"]
        relationships = data["relationships"]
        
        # Process nodes
        x_dict = {}
        x_ids = {}
        for node in nodes:
            node_id = node.get("id")
            node_feature = node.get(node_attr, None)
            if node_feature is None:
                print(f"Node {node_id} has no attribute {node_attr}")
                continue
            node_label = list(node.labels)[0]
            if node_label not in x_dict:
                x_dict[node_label] = []
                x_ids[node_label] = []
            
            x_dict[node_label].append(torch.tensor(node_feature))
            x_ids[node_label].append(node_id)
        
        for node_label, node_features in x_dict.items():
            x_dict[node_label] = torch.vstack(node_features)
            
        # Process relationships
        edge_dict = {}
        
        for rel in relationships:
            if rel.type not in edge_dict:
                edge_dict[rel.type] = [[], []]
            source_id = rel.start_node.get("id")
            target_id = rel.end_node.get("id")
            
            edge_dict[rel.type][0].append(source_id)
            edge_dict[rel.type][1].append(target_id)
        
    return x_dict, x_ids, edge_dict

In [18]:
def normalize_topology(new_idx_to_old, topology):
    # Reverse index mapping based on new idx -> old idx
    old_idx_to_new = dict((v, k) for k, v in new_idx_to_old.items())
    return {rel_type: [[old_idx_to_new[node_id] for node_id in nodes] for nodes in topology] for rel_type, topology in topology.items()}

def create_edge_index(topology):
    edge_index = []
    edge_features = []
    edge_labels = []
    for rel_type, nodes in topology.items():
        src_nodes, dst_nodes = nodes
        edges = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)
        edge_index.append(edges)
        # Extend edge features by one-hot encoding of the relationship type
        edge_features.extend([edge_one_hot[rel_type] for _ in range(len(src_nodes))])
        # Extend edge labels by the relationship type for each edge feature vector
        edge_labels.extend([rel_type for _ in range(len(src_nodes))])
    return torch.cat(edge_index, dim=1), torch.vstack(edge_features), edge_labels

def remap_node_indices(x_ids: dict, edge_dict: dict):
    unique_node_ids = list(set([node_id for id_list in x_ids.values() for node_id in id_list]))
    old_id_to_new = {node_id: i for i, node_id in enumerate(unique_node_ids)}
    new_edge_index_dict = {}
    
    for rel_type, node_ids in edge_dict.items():
        src_nodes, dest_nodes = node_ids
        edge_index = []
        for src, dest in zip(src_nodes, dest_nodes):
            edge_index.append(torch.tensor([old_id_to_new[src], old_id_to_new[dest]], dtype=torch.long))
        new_edge_index_dict[rel_type] = torch.vstack(edge_index).T
        
    return new_edge_index_dict
              
def create_edge_feature_dict(edge_dict):
    edge_feature_dict = {}
    for rel_type, node_ids in edge_dict.items():
        num_edges = len(node_ids[0])
        edge_features = [edge_one_hot[rel_type] for _ in range(num_edges)]
        edge_feature_dict[rel_type] = torch.vstack(edge_features)
        
    return edge_feature_dict

In [19]:
included_nodes = [
    NodeType.PUBLICATION, 
    NodeType.VENUE, 
    NodeType.ORGANIZATION,
    NodeType.AUTHOR,
    NodeType.CO_AUTHOR
]
included_edges = [
    EdgeType.PUB_VENUE,
    EdgeType.VENUE_PUB,
    EdgeType.PUB_ORG,
    EdgeType.ORG_PUB, 
    #EdgeType.SIM_VENUE,
    #EdgeType.SIM_ORG,
    EdgeType.PUB_AUTHOR,
    EdgeType.AUTHOR_PUB,
    EdgeType.AUTHOR_ORG,
    EdgeType.ORG_AUTHOR,
    EdgeType.PUB_ORG,
    EdgeType.ORG_PUB,
]

def sample_triplet(anchor, pos, neg):
    triplet = {}
    for label, node_id in zip(["anchor", "pos", "neg"], [anchor, pos, neg]):
        x_dict, x_ids, edge_dict = fetch_n_hop_neighbourhood(
            start_node_type=NodeType.PUBLICATION, 
            start_node_id=node_id, 
            node_attr="vec",
            node_types=included_nodes,
            edge_types=included_edges,
            max_level=2
        )
        if x_dict is None or len(x_dict.keys()) == 0:
            return None
        if edge_dict is None or len(edge_dict.keys()) == 0:
            return None
            
        edge_index = remap_node_indices(x_ids, edge_dict)
        edge_features = create_edge_feature_dict(edge_dict)
        
        graph = {
            "x_dict": x_dict,
            "x_ids": x_ids,
            "edge_index_dict": edge_index,
            "edge_feature_dict": edge_features,
        }

        triplet[label] = graph
    
    return triplet

In [20]:
class TripleSampler:
    def __init__(self):
        self.data = WhoIsWhoDataset.parse_train()
        
    def iter_triplet_ids_rand(self, num_triplets):
        # yield random triplets from the WhoIsWho dataset
        current_num_triplets = 0
        while current_num_triplets < num_triplets:
            # Get random author
            author_id = random.choice(list(self.data.keys()))
            
            # Check if author has enough data
            normal_data = self.data[author_id]["normal_data"]
            outliers = self.data[author_id]["outliers"]
            if len(normal_data) < 2 or len(outliers) == 0:
                self.data.pop(author_id)
                continue
            # Get random anchor, positive and negative samples, remove anchor
            anchor_id = random.choice(normal_data)
            self.data[author_id]["normal_data"].remove(anchor_id)
            
            pos_sample = random.choice(normal_data)
            neg_sample = random.choice(outliers)
            yield anchor_id, pos_sample, neg_sample
            current_num_triplets += 1
            
    def sample_triplet_ids(self, attempt = 0):
        if attempt > 30:
            print("Unable to sample more triples")
            return None
        # Get random author
        author_id = random.choice(list(self.data.keys()))
        
        # Check if author has enough data
        normal_data = self.data[author_id]["normal_data"]
        outliers = self.data[author_id]["outliers"]
        if len(normal_data) < 2 or len(outliers) == 0:
            self.data.pop(author_id)
            return self.sample_triplet_ids(attempt + 1)
        # Get random anchor, positive and negative samples, remove anchor
        anchor_id = random.choice(normal_data)
        self.data[author_id]["normal_data"].remove(anchor_id)
        
        pos_sample = random.choice(normal_data)
        neg_sample = random.choice(outliers)
        return anchor_id, pos_sample, neg_sample

    def iter_triplets_rand(self, num_triplets):
        # Fetch triplets by id from the neo4j database and yield them
        count = 0
        count_skipped = 0
        while count < num_triplets:                
            anchor_id, pos_id, neg_id = self.sample_triplet_ids()
            triplet = sample_triplet(anchor_id, pos_id, neg_id)
            if triplet is None or type(triplet) is not dict:
                count_skipped += 1
                continue
        
            yield triplet
            count += 1
        #print(f"Requested {num_triplets}, Fetched {count}, Skipped {count_skipped}")

In [21]:
triplet_sampler = TripleSampler()
path = "./data/triplet_dataset"

if not os.path.exists(path):
    os.makedirs(path)
    
# Delete folder contents
for file in os.listdir(path):
    os.remove(os.path.join(path, file))

for batch_id in range(10000):
    file = os.path.join(path, f"triplet_batch_{batch_id}.pt")
    #with open(file, "w") as f:
    data = []
    for triplet in triplet_sampler.iter_triplets_rand(10):
        #print(json.dumps(triplet, indent=2))
        data.append(triplet)
    
    # Save the triplets to disk
    print(f"Saving batch {batch_id} to disk")
    #f.write(json.dumps(data))
    torch.save(data, file)

Saving batch 0 to disk
Saving batch 1 to disk
Saving batch 2 to disk
Saving batch 3 to disk
Saving batch 4 to disk
Saving batch 5 to disk
Saving batch 6 to disk


KeyboardInterrupt: 